In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/clarissaache/Downloads/arcos-fl-statewide-itemized.csv')
df.head

/Users/clarissaache/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,5,13,15,22,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<bound method NDFrame.head of          REPORTER_DEA_NO REPORTER_BUS_ACT                REPORTER_NAME  \
0              PB0034861      DISTRIBUTOR             BELLCO DRUG CORP   
1              PB0034861      DISTRIBUTOR             BELLCO DRUG CORP   
2              PB0034861      DISTRIBUTOR             BELLCO DRUG CORP   
3              PB0034861      DISTRIBUTOR             BELLCO DRUG CORP   
4              PB0034861      DISTRIBUTOR             BELLCO DRUG CORP   
...                  ...              ...                          ...   
15104978       RD0277409      DISTRIBUTOR  MASTERS PHARMACEUTICAL, INC   
15104979       RD0277409      DISTRIBUTOR  MASTERS PHARMACEUTICAL, INC   
15104980       RD0277409      DISTRIBUTOR  MASTERS PHARMACEUTICAL, INC   
15104981       RD0277409      DISTRIBUTOR  MASTERS PHARMACEUTICAL, INC   
15104982       RD0277409      DISTRIBUTOR  MASTERS PHARMACEUTICAL, INC   

           REPORTER_ADDL_CO_INFO       REPORTER_ADDRESS1 REPORTER_ADDRESS2  \
0  

# Columns
A "REPORTER" is an entity that purchases drugs, usually distributors and manufacturers
<br>
A "BUYER" is an entity receiving shipments from reporter. Like retail and online pharmacies, practicitoners other smaller distributors
<br>
**We care about the Buyer columns**

In [3]:
df.shape
# 15104983 obs, 33 columns
df.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'ACTION_INDICATOR', 'ORDER_FORM_NO',
       'CORRECTION_NO', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name',
       'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name',
       'Reporter_family', 'dos_str', 'MME'],
      dtype='object')

# Which columns do we need??
I basically just went through the columns related to buyers one by one checking if the information needed to be cleaned

In [4]:
# Lets get rid of the Reporter columns for now (faster to work with smaller dataset)
cols = ['REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'ACTION_INDICATOR', 'ORDER_FORM_NO',
       'CORRECTION_NO', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name',
       'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name',
       'Reporter_family', 'dos_str', 'MME']
df1 = df[cols]

# Exploring which columns do we care about
I looked at the unique values of the columns that we might be interested in or that might require us to do some cleaning before grouping

In [5]:
df1.DRUG_NAME.unique()
# we only have data for 'OXYCODONE', 'HYDROCODONE'
df1['BUYER_COUNTY'].nunique()
# we have data for 67 counties, which is the amount of counties that exist in FL 
df1['TRANSACTION_CODE'].nunique()
# all transactions are sales (not returns, so we are good here)


1

In [6]:
df1['Measure'].nunique
# they are all 'tab' = tablet

<bound method IndexOpsMixin.nunique of 0           TAB
1           TAB
2           TAB
3           TAB
4           TAB
           ... 
15104978    TAB
15104979    TAB
15104980    TAB
15104981    TAB
15104982    TAB
Name: Measure, Length: 15104983, dtype: object>

# What should we do about the ACTION INDICATOR column?
These may be transactions that were modified, deleted, or adjusted, so basically, we cannot trust the data for these observations. Can we?
There is an important ammount of observations with 'Adjusted' and 'Incert' flags, but only 5 with the 'Delete' flag. Most obs are NaN in this column

In [7]:
df1[df['ACTION_INDICATOR']== 'I']
# This column is an indication of corrected shipments by reporter. 
# Values include A: adjust, D: Delete or I: insert(late-reporter shipment).

,REPORTER_COUNTY,BUYER_DEA_NO,BUYER_BUS_ACT,BUYER_NAME,BUYER_ADDL_CO_INFO,BUYER_ADDRESS1,BUYER_ADDRESS2,BUYER_CITY,BUYER_STATE,BUYER_ZIP,...,DOSAGE_UNIT,TRANSACTION_ID,Product_Name,Ingredient_Name,Measure,MME_Conversion_Factor,Combined_Labeler_Name,Reporter_family,dos_str,MME
3,SUFFOLK,A97118246,RETAIL PHARMACY,"TRU-VALU DRUGS, INC",NaN,101 N. FEDERAL HWY.,NaN,LAKE WORTH,FL,33460,...,300.0,281,OXYCODONE HCL 80MG TABS,OXYCODONE HYDROCHLORIDE,TAB,1.5,"Actavis Pharma, Inc.",Bellco Drug Corp,80.0,32274.00
11,SUFFOLK,AL0185822,RETAIL PHARMACY,LEE-ANN DRUG INC,NaN,DBA-LEE ANN DRUG INC,955 WASHINGTON AVENUE,MIAMI BEACH,FL,33139,...,200.0,1106,HYDROCODONE BITARTRATE AND ACETA 10M,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Par Pharmaceutical,Bellco Drug Corp,10.0,1210.80
15,SUFFOLK,AP2020371,RETAIL PHARMACY,"PEARL STREET PHARMACY, INC",NaN,DBA-BROWNS 8TH STREET PHCY,312 WEST 8TH STREET,JACKSONVILLE,FL,32206,...,500.0,7502,HYDROCODONE BIT. 10MG/ACETAMINOPHEN,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Actavis Pharma, Inc.",Bellco Drug Corp,10.0,3027.00
24,SUFFOLK,BA5362847,RETAIL PHARMACY,ARTHUR'S ORIGINAL PHARMACY,NaN,5816 N UNIVERSITY DRIVE,NaN,TAMARAC,FL,33321,...,300.0,771,OXYCODONE HYDROCHLORIDE CR 20MG TABL,OXYCODONE HYDROCHLORIDE,TAB,1.5,"Actavis Pharma, Inc.",Bellco Drug Corp,20.0,8068.50
25,SUFFOLK,BA7001821,RETAIL PHARMACY,ACOLOGY PRESCRIPTION COMPOUNDING,NaN,131 DEL PRADO BLVD SOUTH,SUITE 101 &102,CAPE CORAL,FL,33990,...,200.0,7151,OXYCOD.HCL ER TABS;20MG/TAB,OXYCODONE HYDROCHLORIDE,TAB,1.5,Par Pharmaceutical,Bellco Drug Corp,20.0,5379.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15061990,MADISON,BB7536595,RETAIL PHARMACY,BELLEVIEW COMMUNITY PHARMACY,NaN,10762 SE HIGHWAY 441,NaN,BELLEVIEW,FL,34420,...,500.0,712069477,OXYCODONE HCL 30MG USP TABLETS,OXYCODONE HYDROCHLORIDE,TAB,1.5,Par Pharmaceutical,Cardinal Health,30.0,20171.25
15104213,ORANGE,AD3267033,PRACTITIONER,"DEAN, RANDY M MD",NaN,4300 UNIVERSITY DRIVE,NaN,LAUDERHILL,FL,33351,...,240.0,1313,HYDROCODONE 5MG/APAP 500 TABS,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Dispensing Solutions Inc.,Dispensing Solutions,5.0,726.48
15104214,ORANGE,AD3267033,PRACTITIONER,"DEAN, RANDY M MD",NaN,4300 UNIVERSITY DRIVE,NaN,LAUDERHILL,FL,33351,...,240.0,1314,HYDROCODONE 5MG/APAP 500 TABS,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Dispensing Solutions Inc.,Dispensing Solutions,5.0,726.48
15104262,ORANGE,BR4765042,PRACTITIONER,"RADECKI, RICHARD",SHRINERS HOSPITAL FOR CHILDREN - TAMPA UNIT,500 HARBOUR POST DR,NaN,TAMPA,FL,33602,...,720.0,36,HYDROCODONE/APAP 5MG/500MG TABLETS,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Dispensing Solutions Inc.,Dispensing Solutions,5.0,2179.44


In [8]:
df1['BUYER_NAME'].nunique()
# there are 9566 different buyers!

9566

# Subset 
Now that we know more about the columns we actually care about, I created an even smaller subset

In [9]:
df2 = df[['BUYER_NAME', 'BUYER_COUNTY',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'Ingredient_Name','MME_Conversion_Factor', 'dos_str']]
df2.head

<bound method NDFrame.head of                    BUYER_NAME  BUYER_COUNTY    DRUG_NAME  QUANTITY  UNIT  \
0         TRU-VALU DRUGS, INC    PALM BEACH    OXYCODONE       1.0   NaN   
1         TRU-VALU DRUGS, INC    PALM BEACH    OXYCODONE       8.0   NaN   
2         TRU-VALU DRUGS, INC    PALM BEACH    OXYCODONE       3.0   NaN   
3         TRU-VALU DRUGS, INC    PALM BEACH    OXYCODONE       3.0   NaN   
4         TRU-VALU DRUGS, INC    PALM BEACH    OXYCODONE       6.0   NaN   
...                       ...           ...          ...       ...   ...   
15104978       FELKY RX  LLC.  HILLSBOROUGH    OXYCODONE     100.0   NaN   
15104979       FELKY RX  LLC.  HILLSBOROUGH  HYDROCODONE       1.0   NaN   
15104980       FELKY RX  LLC.  HILLSBOROUGH  HYDROCODONE       5.0   NaN   
15104981       FELKY RX  LLC.  HILLSBOROUGH    OXYCODONE      25.0   NaN   
15104982       FELKY RX  LLC.  HILLSBOROUGH    OXYCODONE      48.0   NaN   

          STRENGTH  TRANSACTION_DATE  CALC_BASE_WT_IN_GM 

# Clean the columns we need
Now that we have a smaller subset of the data with the columns we may need, let's see if the data is complete

In [10]:
df2['BUYER_COUNTY'].isna().sum()
# 16 observations do not have a county
df2['BUYER_COUNTY'] = df['BUYER_COUNTY'].replace(np.nan, 'Unknown')
#replaced with Unknown

/var/folders/wj/g21k_l5x5l95gccmpm_gvn0c0000gn/T/ipykernel_50170/519255045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BUYER_COUNTY'] = df['BUYER_COUNTY'].replace(np.nan, 'Unknown')


In [11]:
df2['TRANSACTION_DATE'].isna().sum()
# AMAZING
# All transactions have dates, but they are in this weird format

0

In [19]:
# changing dates that look like 8252007 to date format
df2['T_DATE'] = pd.to_datetime(df2['TRANSACTION_DATE'], format='%m%d%Y', errors='coerce')
df2['T_DATE']=pd.DatetimeIndex(df2['T_DATE']).year
df_by_county = df2.groupby(['BUYER_COUNTY', 'T_DATE']).sum().reset_index()
df_by_county['MME']=df_by_county['CALC_BASE_WT_IN_GM']*1000*df_by_county['MME_Conversion_Factor']
finalcolumns=['BUYER_COUNTY', 'T_DATE', 'MME']
df_final=df_by_county[finalcolumns]
# Quick sense check: looks like the TRANSACTION DATE and new T DATE columns match!
df2

/var/folders/wj/g21k_l5x5l95gccmpm_gvn0c0000gn/T/ipykernel_50170/1596842313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['T_DATE'] = pd.to_datetime(df2['TRANSACTION_DATE'], format='%m%d%Y', errors='coerce')
/var/folders/wj/g21k_l5x5l95gccmpm_gvn0c0000gn/T/ipykernel_50170/1596842313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['T_DATE']=pd.DatetimeIndex(df2['T_DATE']).year


,BUYER_NAME,BUYER_COUNTY,DRUG_NAME,QUANTITY,UNIT,STRENGTH,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,Ingredient_Name,MME_Conversion_Factor,dos_str,T_DATE,mg_of_drug
0,"TRU-VALU DRUGS, INC",PALM BEACH,OXYCODONE,1.0,NaN,0.0,8182006,3.5860,100.0,OXYCODONE HYDROCHLORIDE,1.5,40.0,2006,4000.0
1,"TRU-VALU DRUGS, INC",PALM BEACH,OXYCODONE,8.0,NaN,0.0,11292006,7.1720,800.0,OXYCODONE HYDROCHLORIDE,1.5,10.0,2006,8000.0
2,"TRU-VALU DRUGS, INC",PALM BEACH,OXYCODONE,3.0,NaN,0.0,2062007,21.5160,300.0,OXYCODONE HYDROCHLORIDE,1.5,80.0,2007,24000.0
3,"TRU-VALU DRUGS, INC",PALM BEACH,OXYCODONE,3.0,NaN,0.0,3012007,21.5160,300.0,OXYCODONE HYDROCHLORIDE,1.5,80.0,2007,24000.0
4,"TRU-VALU DRUGS, INC",PALM BEACH,OXYCODONE,6.0,NaN,0.0,4162007,5.3790,600.0,OXYCODONE HYDROCHLORIDE,1.5,10.0,2007,6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15104978,FELKY RX LLC.,HILLSBOROUGH,OXYCODONE,100.0,NaN,0.0,12042008,268.9500,10000.0,OXYCODONE HYDROCHLORIDE,1.5,30.0,2008,300000.0
15104979,FELKY RX LLC.,HILLSBOROUGH,HYDROCODONE,1.0,NaN,0.0,8132009,3.0270,500.0,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,1.0,10.0,2009,5000.0
15104980,FELKY RX LLC.,HILLSBOROUGH,HYDROCODONE,5.0,NaN,0.0,12302009,1.5135,500.0,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,1.0,5.0,2009,2500.0
15104981,FELKY RX LLC.,HILLSBOROUGH,OXYCODONE,25.0,NaN,0.0,3182010,67.2375,2500.0,OXYCODONE HYDROCHLORIDE,1.5,30.0,2010,75000.0


In [13]:
# did it work alright?
df2['T_DATE'].isnull().any()
# YES!

False

# Grouping
Looks like each row is a transaction, so in order to get how much Oxi and Hydroxi is available to be prescribed in each county, we need to collaps those quantities into one row per county... but we also care about the date

In [14]:
# Created a new total quantity column because I am not sure the "CALC_BASE_WT_IN_GM" 
# (which is the total active weight) is really a measure of the quantity of the druge avaiable
df2['mg_of_drug']=df2.DOSAGE_UNIT*df2.dos_str

/var/folders/wj/g21k_l5x5l95gccmpm_gvn0c0000gn/T/ipykernel_50170/3305587141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['mg_of_drug']=df2.DOSAGE_UNIT*df2.dos_str


In [20]:
df_by_county

,BUYER_COUNTY,T_DATE,QUANTITY,UNIT,STRENGTH,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME_Conversion_Factor,dos_str,mg_of_drug,MME
0,ALACHUA,2006,54294.0,0.0,0.0,143996844658,60803.410442,7333056.0,27070.0,279318.9940,7.701488e+07,1.645948e+12
1,ALACHUA,2007,56118.0,0.0,0.0,150966302856,69845.182455,8139364.0,28245.5,289063.3620,8.807775e+07,1.972812e+12
2,ALACHUA,2008,64869.0,0.0,0.0,163410570312,83284.874966,9100379.0,30504.0,325939.2345,1.038755e+08,2.540522e+12
3,ALACHUA,2009,73351.0,0.0,0.0,167226992120,101391.418612,10139373.0,31367.5,344488.2230,1.246617e+08,3.180395e+12
4,ALACHUA,2010,74339.0,0.0,0.0,160772788740,107176.406164,10072070.0,30733.0,338714.5390,1.301124e+08,3.293852e+12
...,...,...,...,...,...,...,...,...,...,...,...,...
597,WASHINGTON,2010,9853.0,0.0,2890.0,20069835880,12099.173792,1773389.0,3442.0,32754.0130,1.771627e+07,4.164536e+10
598,WASHINGTON,2011,9622.0,0.0,0.0,21119611979,12072.211013,1849720.0,3595.0,32100.8550,1.790832e+07,4.339960e+10
599,WASHINGTON,2012,10189.0,0.0,0.0,21662724736,13330.677317,1963630.0,3809.0,34519.3745,1.952295e+07,5.077655e+10
600,WASHINGTON,2013,9443.0,0.0,0.0,21034922703,13648.062362,1953750.0,3668.5,33760.3615,1.978895e+07,5.006792e+10


In [22]:
df_final
df_final["State"]='Florida'
# MME is the Morphine Milligram Equivalent of the Oxycodone and Hydroxicodone, or how the specific drug compares to a morphine equivilent.

/var/folders/wj/g21k_l5x5l95gccmpm_gvn0c0000gn/T/ipykernel_50170/4108400332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["State"]='Florida'


In [25]:
df_final_FL = df_final
df_final_FL

,BUYER_COUNTY,T_DATE,MME,State
0,ALACHUA,2006,1.645948e+12,Florida
1,ALACHUA,2007,1.972812e+12,Florida
2,ALACHUA,2008,2.540522e+12,Florida
3,ALACHUA,2009,3.180395e+12,Florida
4,ALACHUA,2010,3.293852e+12,Florida
...,...,...,...,...
597,WASHINGTON,2010,4.164536e+10,Florida
598,WASHINGTON,2011,4.339960e+10,Florida
599,WASHINGTON,2012,5.077655e+10,Florida
600,WASHINGTON,2013,5.006792e+10,Florida
